In [1]:
#### Use langchain multiple tools by agent and action, also show config file
###https://medium.com/@astropomeai/advanced-multi-agent-ai-system-implementing-iterative-processing-feedback-loops-and-evaluation-b9cccfc4c9d1
from langchain import OpenAI, LLMMathChain, SerpAPIWrapper
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.tools import BaseTool
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langchain_anthropic import ChatAnthropic
from langchain_core.tools import tool
#from langgraph.checkpoint import Checkpointer
from langgraph.checkpoint.memory import MemorySaver
from typing import List, TypedDict, Annotated, Literal
import operator

from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from pydantic import BaseModel
from langchain.schema import HumanMessage, AIMessage
import json, os
import requests
import pandas as pd
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.checkpoint.memory import MemorySaver
from langchain.memory import ConversationBufferMemory
import uuid
from typing import Literal, Dict, List
memory = MemorySaver()
API_KEY = ''
from langchain.tools import tool
os.environ['OPENAI_API_KEY']=API_KEY
def create_llm(model_class, model_name, temperature=0.7):
    return model_class(model=model_name, temperature=temperature)
# Initialize LLM models
#claude = create_llm(ChatAnthropic, "claude-3-haiku-20240307")
#gemini = create_llm(ChatGoogleGenerativeAI, "gemini-1.5-flash-latest")
#deepseek = ChatOpenAI(model="deepseek-chat", temperature=0, openai_api_key=Deepseek_key, openai_api_base='https://api.deepseek.com')
openai = create_llm(ChatOpenAI, "gpt-4o")
reviewer = create_llm(ChatOpenAI, "gpt-4o")

from langgraph.graph import END, StateGraph
from typing import List, Tuple


In [28]:
# AML Hypothesis Generation Multi-Agent System using LangGraph and Streamlit

from typing import List, Tuple, Literal, Annotated
from typing_extensions import TypedDict
import pandas as pd
from langchain.schema import HumanMessage, AIMessage
from langchain.chat_models import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


### new ###
# === Define AgentState === #
class AgentState(TypedDict):
    research_topic: Annotated[str, add_messages]  # <-- Allow multiple writes
    expectation: Annotated[str, add_messages]
    expected_result: Annotated[str, add_messages]
    
    #expected_result: str
    feedback: Annotated[str, add_messages]
    evaluation: Annotated[int, operator.and_] 
    #iteration: int
    iteration: Annotated[int, operator.and_]
    continue_loop: Annotated[int, operator.and_]
    messages: Annotated[List[HumanMessage], add_messages]
    agent_outputs: Annotated[List[AIMessage], add_messages]


# === Supervisor Node === #
class SupervisorAgent:
    def __init__(self):
        self.name = "Supervisor"

    def __call__(self, state):
        print("Supervisor")
        if state.get('messages'):
            topic = state['messages'][0].content
            state['research_topic'] = topic
            state['expected_result'] = (
                f"Provide an in-depth and testable hypothesis for: {topic}, focusing on AML treatment strategies."
            )
        else:
            state['research_topic'] = "Default AML research topic"
            state['expected_result'] = "Provide a default AML hypothesis."
        return state


# === Hypothesis Generation Agent === #
class HypothesisGenerationAgent:
    def __init__(self, llm, agent_name="", gene_file=None, custom_prompt=None, custom_topic=None, custom_expectation=None):
        self.llm = llm
        self.agent_name = agent_name
        self.gene_file = gene_file
        self.custom_prompt = custom_prompt
        self.custom_topic = custom_topic
        self.custom_expectation = custom_expectation
        self.gene_list = self.load_genes() if gene_file else []

    def load_genes(self):
        try:
            df = pd.read_csv(self.gene_file)
            return df['x_data'].dropna().tolist()
        except Exception as e:
            print(f"Error loading gene list for {self.agent_name}: {e}")
            return []

    def __call__(self, state):
        print("Hypothesis")
        topic = self.custom_topic or state.get("research_topic", "")
        expectation = self.custom_expectation or state.get("expectation", "")

        gene_context = f"The following genes interact: {', '.join(self.gene_list)}." if self.gene_list else ""

        prompt = self.custom_prompt.format(
            agent_name=self.agent_name,
            research_topic=topic,
            expectation=expectation,
            gene_context=gene_context
        ) if self.custom_prompt else (
            f"You are {self.agent_name}, an expert in hypothesis generation.\n"
            f"Research Topic: {topic}\nExpectation: {expectation}\n{gene_context}\n"
            "Formulate a novel, testable hypothesis."
        )

        response = self.llm(messages=[HumanMessage(role="user", content=prompt)])
        output = AIMessage(role="assistant", content=response.content)
        state['agent_outputs'].append(output)
        return state


# === Reviewer Node === #
class ReviewerNode:
    def __init__(self, llm):
        self.name = "Reviewer"
        self.llm = llm

    def __call__(self, state):
        print("Reviewer")
        expected_result = state.get('expected_result', '')
        agent_outputs = state.get('agent_outputs', [])
        if not agent_outputs:
            state['feedback'] = "The agent output is missing."
            return state

        outputs_text = "\n".join([f"{msg.role}: {msg.content}" for msg in agent_outputs])

        review_prompt = (
            f"Expected result:\n{expected_result}\n\nAgent outputs:\n{outputs_text}\n\n"
            "Compare the outputs to the expected result and give feedback and an evaluation score between 1 and 10.\n"
            "Format:\nFeedback: ...\nEvaluation Score: X"
        )

        response = self.llm(messages=[HumanMessage(role="user", content=review_prompt)])
        content = response.content if hasattr(response, 'content') else str(response)
        feedback, evaluation = self.parse_feedback_and_evaluation(content)
        state['feedback'] = feedback
        state['evaluation'] = evaluation
        return state

    def parse_feedback_and_evaluation(self, content):
        lines = content.splitlines()
        feedback = ''
        evaluation = 0
        for line in lines:
            if line.startswith("Feedback:"):
                feedback = line.replace("Feedback:", "").strip()
            elif line.startswith("Evaluation Score:"):
                try:
                    evaluation = int(line.replace("Evaluation Score:", "").strip())
                except ValueError:
                    evaluation = 0
        return feedback, evaluation


# === Loop Control Node === #
class LoopControlNode:
    def __init__(self, max_iterations):
        self.max_iterations = max_iterations

    def __call__(self, state):
        print("Loop Control")
        state['iteration'] += 1
        state['continue_loop'] = state['iteration'] < self.max_iterations
        return state




# === Gene files, prompts, and goals === #
gene_files = {
    "agent1": "genes_agent1.csv",
    "agent2": "genes_agent2.csv",
    "agent3": "genes_agent3.csv",
    "agent4": "genes_agent4.csv",
    "agent5": "genes_agent5.csv"
}

agent_goals = {
    "agent1": {
        "research_topic": "Gene network perturbation in AML",
        "expectation": "Identify genes whose disruption can reverse downregulated metabolic pathways in AML cells."
    },
    "agent2": {
        "research_topic": "ER stress and UPR in AML",
        "expectation": "Propose a targetable node in the UPR pathway to restore apoptotic signaling."
    },
    "agent3": {
        "research_topic": "Immune evasion in AML",
        "expectation": "Identify how to reverse T-cell suppression in the AML microenvironment."
    },
    "agent4": {
        "research_topic": "Drug repurposing for Myc-driven AML",
        "expectation": "Identify a non-cancer drug that suppresses Myc-related signaling in AML."
    },
    "agent5": {
        "research_topic": "TF reprogramming in AML",
        "expectation": "Show how transcription factors can re-activate differentiation programs in AML."
    }
}

custom_prompts = {
    "agent1": (
        "You are {agent_name}, a systems biologist.\n"
        "Topic: {research_topic}\nExpectation: {expectation}\n{gene_context}\n"
        "Suggest how perturbing one or more of these genes could reverse AML progression."
    ),
    "agent2": (
        "You are {agent_name}, an ER stress specialist.\nTopic: {research_topic}\nExpectation: {expectation}\nFormulate a hypothesis about UPR pathways in AML."
    ),
    "agent3": (
        "You are {agent_name}, focused on immune escape in AML.\nTopic: {research_topic}\nExpectation: {expectation}\nIdentify a way to restore T-cell activation."
    ),
    "agent4": (
        "You are {agent_name}, a drug repurposing analyst.\nTopic: {research_topic}\nExpectation: {expectation}\nSuggest a non-oncology drug to block Myc."
    ),
    "agent5": (
        "You are {agent_name}, a molecular biologist.\nTopic: {research_topic}\nExpectation: {expectation}\nPropose how TFs can restore differentiation in AML."
    )
}


# === Build Graph === #
openai = ChatOpenAI(model="gpt-4o", openai_api_key=API_KEY)

graph_builder = StateGraph(AgentState)
graph_builder.add_node("supervisor", SupervisorAgent())
graph_builder.add_node("reviewer", ReviewerNode(openai))
graph_builder.add_node("loop_control", LoopControlNode(max_iterations=5))

for i in range(1, 6):
    agent = f"agent{i}"
    graph_builder.add_node(agent, HypothesisGenerationAgent(
        openai,
        agent_name=agent,
        gene_file=gene_files[agent],
        custom_prompt=custom_prompts[agent],
        custom_topic=agent_goals[agent]["research_topic"],
        custom_expectation=agent_goals[agent]["expectation"]
    ))

# Define edges
graph_builder.add_edge(START, "supervisor")
for i in range(1, 6):
    graph_builder.add_edge("supervisor", f"agent{i}")
    graph_builder.add_edge(f"agent{i}", "reviewer")

graph_builder.add_edge("reviewer", "loop_control")
graph_builder.add_conditional_edges("loop_control", lambda state: "supervisor" if state['continue_loop'] else END)

# Compile
graph = graph_builder.compile()

# === Initial State === #
initial_state = {
    "research_topic": "Suggest a drug target for AML",
    "expectation": "Drug should suppress AML progression using validated pathways",
    "expected_result": "A high-quality hypothesis focused on drug repurposing for AML",
    "feedback": "",
    "evaluation": 0,
    "iteration": 0,
    "continue_loop": True,
    "messages": [HumanMessage(role="user", content="Suggest a repurposed drug that inhibits AML progression.")],
    "agent_outputs": []
}

# === Run === #
final_state = graph.invoke(initial_state)

# === Print Outputs === #
print("\n=== Final Hypotheses and Feedback ===\n")
for msg in final_state['agent_outputs']:
    print(f"{msg.role} ->\n{msg.content}\n")

print("\nFeedback:", final_state['feedback'])
print("Evaluation Score:", final_state['evaluation'])


Supervisor
Hypothesis
Hypothesis
Hypothesis
Hypothesis
Hypothesis
Reviewer
Loop Control

=== Final Hypotheses and Feedback ===

assistant ->
In the context of drug repurposing for Myc-driven acute myeloid leukemia (AML), one promising non-oncology drug that has shown potential in suppressing Myc-related signaling is Metformin. Originally used as a first-line treatment for type 2 diabetes, Metformin has attracted interest for its potential anticancer properties, including its ability to impact Myc-driven pathways.

Metformin is known to activate AMP-activated protein kinase (AMPK), a crucial energy sensor in cells. Activation of AMPK can lead to the inhibition of the mTOR pathway, which is often upregulated in cancer cells, including those driven by Myc overexpression. By modulating these pathways, Metformin can indirectly impact Myc activity and reduce the growth and survival of Myc-driven cancer cells.

Preclinical studies have demonstrated that Metformin can reduce Myc expression and

In [1]:
# Normalize output to standard dictionary format with 'role' and 'content' keys
def normalize_output(output):
    if hasattr(output, 'role') and hasattr(output, 'content'):
        return {"role": output.role, "content": output.content}
    elif isinstance(output, dict):
        return {"role": output.get('role', 'unknown'), "content": output.get('content', '')}
    elif hasattr(output, 'content'):  # Custom object handling
        return {"role": "assistant", "content": output.content}
    else:
        return {"role": "unknown", "content": str(output)}


# Print final outputs after normalizing the output format
print("Final Agent Outputs:")
for output in final_state['agent_outputs']:
    normalized = normalize_output(output)
    print(f"{normalized['role']}:\n{normalized['content']}\n")


Final Agent Outputs:


NameError: name 'final_state' is not defined

In [3]:
import json

# Helper function to normalize outputs
def normalize_output(output):
    if hasattr(output, 'role') and hasattr(output, 'content'):
        return {"role": output.role, "content": output.content}
    elif isinstance(output, dict):
        return {"role": output.get('role', 'unknown'), "content": output.get('content', '')}
    elif hasattr(output, 'content'):  # Handle custom output objects
        return {"role": "assistant", "content": output.content}
    else:
        return {"role": "unknown", "content": str(output)}

# Save results to a text file
def save_results_to_text(final_state, filename="final_results.txt"):
    with open(filename, "w") as file:
        file.write("🎯 ===== Final Results After All Iterations =====\n")
        file.write(f"Total Iterations Run: {final_state['iteration']}\n")
        file.write(f"Final Feedback: {final_state.get('feedback', 'No feedback')}\n")
        file.write(f"Final Evaluation Score: {final_state.get('evaluation', 0.0)}\n\n")

        final_hypotheses = final_state.get('hypotheses', [])
        if final_hypotheses:
            file.write("Final Hypotheses:\n")
            for hypothesis in final_hypotheses:
                file.write(f"- Topic: {hypothesis['Topic']}\n")
                file.write(f"  Hypothesis: {hypothesis['Hypothesis']}\n\n")

# Save results to a JSON file
def save_results_to_json(final_state, filename="final_results.json"):
    with open(filename, "w") as json_file:
        json.dump(final_state, json_file, indent=4)

# Updated run_graph function with saving feature
def run_graph(graph, initial_state, max_iterations=15):
    state = initial_state

    for iteration in range(max_iterations):
        print(f"\n🔄===== Iteration {iteration + 1} =====\n")
        
        state = graph.invoke(state, {"recursion_limit": 15})

        # Print Expected Result
        print(f"Expected Result:\n{state.get('expected_result', 'No expected result provided.')}\n")

        # Print Generated Hypotheses
        hypotheses = state.get('hypotheses', [])
        if hypotheses:
            print("Generated Hypotheses:")
            for hypothesis in hypotheses:
                print(f"- Topic: {hypothesis['Topic']}")
                print(f"  Hypothesis: {hypothesis['Hypothesis']}\n")
        else:
            print("No hypotheses generated in this iteration.\n")

        # Print Agent Outputs using normalized output format
        agent_outputs = state.get('agent_outputs', [])
        if agent_outputs:
            print("Agent Outputs:")
            for output in agent_outputs:
                normalized = normalize_output(output)
                print(f"{normalized['role']}:\n{normalized['content']}\n")
        else:
            print("No agent outputs yet.\n")

        # Print Feedback and Evaluation Score
        print(f"Feedback: {state.get('feedback', 'No feedback')}")
        print(f"Evaluation Score: {state.get('evaluation', 0.0)}\n")
        
        if not state.get('continue_loop', False):
            print("\n✅ Workflow completed. No further iterations required.\n")
            break

    # Save results after completion
    save_results_to_text(state, "final_results.txt")
    save_results_to_json(state, "final_results.json")

    # Print final results
    print("\n🎯 ===== Final Results After All Iterations =====\n")
    print(f"Total Iterations Run: {state['iteration']}")
    print(f"Final Feedback: {state.get('feedback', 'No feedback')}")
    print(f"Final Evaluation Score: {state.get('evaluation', 0.0)}")

    final_hypotheses = state.get('hypotheses', [])
    if final_hypotheses:
        print("\nFinal Hypotheses:")
        for hypothesis in final_hypotheses:
            print(f"- Topic: {hypothesis['Topic']}")
            print(f"  Hypothesis: {hypothesis['Hypothesis']}\n")

    print("\n📁 Results have been saved to 'final_results.txt' and 'final_results.json'.")
